In [15]:
from src.data import OxfordPetDatasetLoader
from src.config import Config
from src.data import DataPreprocessor, DataAugmentor


config = Config()
config.USE_IMAGENET_NORM = True          # For transfer learning
config.PRESERVE_ASPECT_RATIO = True      # Better image quality
config.ENABLE_QUALITY_ENHANCEMENT = True # Image enhancement
config.NORMALIZATION_METHOD = 'imagenet' # Best for pretrained models

In [16]:

loader = OxfordPetDatasetLoader(
    data_dir="./data",  # Tùy chọn
    download=True,
    log_level='INFO'
)

INFO:src.data.dataset_loader:Initialized Oxford Pet Dataset Loader
INFO:src.data.dataset_loader:Data directory: ./data


In [17]:

train_ds, val_ds, test_ds = loader.create_train_val_test_splits(
    val_split=0.2,
    seed=42
)

INFO:src.data.dataset_loader:Creating train/validation/test splits...
INFO:src.data.dataset_loader:Checking dataset availability...
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Dataset already available.
INFO:src.data.dataset_loader:Loading raw dataset splits: ['train', 'test']
INFO:absl:Load dataset info from data/oxford_iiit_pet/4.0.0
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Creating a tf.data.Dataset reading 4 files located in folders: data/oxford_iiit_pet/4.0.0.
INFO:absl:Constructing tf.data.Dataset oxford_iiit_pet for split ['train', 'test'], from data/oxford_iiit_pet/4.0.0
INFO:src.data.dataset_loader:Train set: 3,680 examples
INFO:src.data.dataset_loader:Test set: 3,669 examples
INFO:src.data.dataset_loader:Split sizes - Train: 2,944, Val: 736, Test: 3,669


In [18]:
cfg = Config()

In [19]:
train_ds.element_spec

{'file_name': TensorSpec(shape=(), dtype=tf.string, name=None),
 'head_bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None),
 'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None),
 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [21]:
import tensorflow as tf

preprocessor = DataPreprocessor(config=cfg)
train_ds = train_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocessor.preprocess_sample, num_parallel_calls=tf.data.AUTOTUNE)

augmentor = DataAugmentor(config=cfg, target_height=cfg.IMG_SIZE[0], target_width=cfg.IMG_SIZE[1])
train_ds = train_ds.map(augmentor, num_parallel_calls=tf.data.AUTOTUNE)

   

In [22]:
def format_detection(sample):
    image = sample['image']
    target = {
        'bbox': sample['head_bbox'],
        'label': sample['label'],
        'species': sample['species'],
            }
    return image, target
    
output_signature = (
            tf.TensorSpec(shape=(cfg.IMG_SIZE[0], cfg.IMG_SIZE[1], 3), dtype=tf.float32),
            {
                'bbox': tf.TensorSpec(shape=(4,), dtype=tf.float32),
                'label': tf.TensorSpec(shape=(), dtype=tf.int64),
                'species': tf.TensorSpec(shape=(), dtype=tf.int64),
            }
        )
train_ds = train_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(format_detection, num_parallel_calls=tf.data.AUTOTUNE)

In [23]:
train_ds.element_spec

(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 {'bbox': TensorSpec(shape=(4,), dtype=tf.float32, name=None),
  'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
  'species': TensorSpec(shape=(), dtype=tf.int64, name=None)})

In [26]:
# Get the sample
sample = next(iter(train_ds))
sample


(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[-1.058022  ,  3.2451658 ,  4.184864  ],
         [-0.63337547,  3.6889808 ,  4.6247435 ],
         [-0.5866383 ,  3.737828  ,  4.673157  ],
         ...,
         [-5.665044  , -5.946573  , -5.3901305 ],
         [-6.129087  , -6.546707  , -5.946902  ],
         [-6.52969   , -7.084718  , -6.440719  ]],
 
        [[-0.36763743,  3.966716  ,  4.900014  ],
         [ 0.02506831,  4.377149  ,  5.306808  ],
         [-0.01225097,  4.282506  ,  5.2130036 ],
         ...,
         [-4.90237   , -5.1037893 , -4.4149256 ],
         [-5.3748207 , -5.727349  , -4.971697  ],
         [-5.839305  , -6.231978  , -5.465514  ]],
 
        [[ 0.03831434,  4.276551  ,  5.320529  ],
         [ 0.42490277,  4.68059   ,  5.7209854 ],
         [ 0.37148815,  4.607006  ,  5.6480546 ],
         ...,
         [-4.912265  , -4.9909406 , -4.2195754 ],
         [-5.3398867 , -5.570451  , -4.728241  ],
         [-5.7711415 , -6.0211725 , -5.174966 